In [28]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import modeldefine
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [112]:
alpha = 0.01

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def calculate_v(contextinfo_list, A, theta):
    vj_list = []
    feature_list = []
    for i in contextinfo_list:
        feature = model(i.to(device)).cpu()
        first_item =  torch.mm( feature.view(1,-1) , theta)
        second_item = alpha * UCB(A, feature)
        vj_list.append((first_item + second_item).item())
        feature_list.append(feature.detach().numpy())
    return np.array(vj_list), feature_list

def update_A(A, info_subset):
    for i in info_subset:
        i = torch.tensor(i, dtype=torch.float32,device=device)
        feature = model(i.to(device)).view(1,-1).cpu()
        A = A + torch.mm(feature.t(), feature)
    return A

def prob(vj_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return [np.exp(vj_list[i]) / sum for i in range(len(vj_list))]  

def revenue(vj_list, reward_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return np.sum(np.multiply(np.exp(vj_list), reward_list) / sum)

def assort(contextinfo_list, reward_list, vj_list):
    length = len(vj_list)
    # sort the contextinfo_list and vj with descending order of reward_list
    sorted_list = sorted(zip(contextinfo_list, vj_list, reward_list), key=lambda x: x[2], reverse=True)
    contextinfo_list = [x[0] for x in sorted_list]
    vj_list = [x[1] for x in sorted_list]
    reward_list = [x[2] for x in sorted_list]

    # calculate the optimal assortment
    optimal_assort = []
    optimal_reward = 0
    index = 1
    for i in range(length):
        if revenue(vj_list[:index], reward_list[:index]) >= optimal_reward:
            optimal_reward = revenue(vj_list[:index], reward_list[:index])
            index += 1
        else:
            break
    return contextinfo_list[:index]

# this is for the linear purchase model when v = x dot theta
def get_linear_purchase(feature_list):
    true_Vlist = [(TRUE_THETA @ feature_list[i].reshape(-1,1)).item() for  i in range(len(feature_list))]
    prob_list = prob(true_Vlist)

    # sample item according to prob_list
    if random.uniform(0,1) < 1 - np.sum(prob_list):
        return np.array([0 for i in range(len(feature_list))])
    else:
        returnlist = [0 for i in range(len(feature_list))]
        indexchoose = random.choices([i for i in range(len(prob_list))], weights = prob_list)[0]
        returnlist[indexchoose] = 1
        return np.array(returnlist)

In [3]:
def likelihood(theta, feature_list,y_list):
    # feature's dimension is len * dimension , theta is 1*dimension
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    ln_prob = np.log(prob(v_list))
    summation = ln_prob * y_list
    return -1 * np.sum(ln_prob)

def likelihood_derivative(theta, feature_list, y_list):
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    prob_list = prob(v_list)
    summation = np.matmul(np.array(feature_list).T, (y_list - prob_list))
    return -1 * summation

In [4]:
import numpy as np
from scipy.optimize import minimize

initial_guess = np.array([0.5 for i in range(15)])  

known_feature = a
known_y =  y

result = minimize(likelihood, initial_guess, args=(known_feature, known_y), method='SLSQP', 
                  constraints={'type':'eq', 'fun': likelihood_derivative, 'args':(known_feature, known_y)})

optimal_parameters = result.x
print("最优参数:", optimal_parameters)


NameError: name 'a' is not defined

In [5]:
class CustomLikelihoodLoss(nn.Module):
    def __init__(self, theta):
        super(CustomLikelihoodLoss, self).__init__()
        self.theta = theta
        
    def forward(self, output_list, y_list):
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device)
            v = torch.mm(output, self.theta.view(-1,1))
            prob = torch.exp(v) / (torch.sum(torch.exp(v)) + 1)
            loss += torch.sum(torch.log(prob) * y)
            index += 1  
        return -loss

In [100]:
# construct a test set 
a = torch.randn(100,10).numpy()
theta  = np.random.randint(1,4,(1,10)).reshape(-1)
y = np.array([1] + [0 for i in range(99)])
y = np.random.permutation(y)

a_list = []
y_list = []
for i in [20,30,50,100, 60, 80]:
    a = torch.randn(i,10).numpy()
    a_list.append(a)
    y = np.array([1] + [0 for i in range(i-1)]) 
    y = np.random.permutation(y)
    y_list.append(y)


In [102]:
import modeldefine
import torch.optim as optim
model = modeldefine.Model(5,10,10,5).to(device)
# 10 20 20 20 20 20  5
optimizer = optim.Adam(model.parameters(), lr=0.01)


tensor([ 5, 10, 10, 10, 10, 10, 10], dtype=torch.int32)


In [103]:
loss_function = CustomLikelihoodLoss(torch.Tensor(theta).to(device))
epochs = 10

for epoch in range(epochs):
    output_list = [model(torch.tensor(a).to(device)) for a in a_list]
    loss = loss_function(output_list, y_list)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
      
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch [1/10], Loss: 2288.6337890625
Epoch [2/10], Loss: 1637.2301025390625
Epoch [3/10], Loss: 1473.3623046875
Epoch [4/10], Loss: 1453.34814453125
Epoch [5/10], Loss: 1440.3856201171875
Epoch [6/10], Loss: 1431.3837890625
Epoch [7/10], Loss: 1426.150146484375
Epoch [8/10], Loss: 1421.9503173828125
Epoch [9/10], Loss: 1418.9248046875
Epoch [10/10], Loss: 1417.478271484375


In [7]:
CONTEXT_ARRAY = np.load('linear_data/features.npy') 
REWARD_ARRAY = np.load('linear_data/rewards.npy')
TRUE_THETA = np.load('linear_data/theta.npy')

In [9]:
data_length = len(CONTEXT_ARRAY)

# define the hyperparameters
input_size = 20
hidden_size = 20
output_size = 10
num_layers = 10

beta = 0.1
lambd = 1
H = 100

# initialize the parameters

theta = np.random.randn(output_size, 1) / np.sqrt(output_size)
theta = torch.tensor(theta, dtype=torch.float32)
LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)

for t in range(0,10000):
    context = CONTEXT_ARRAY[t]
    profit = REWARD_ARRAY[t]
    v_array = calculate_v(torch.tensor(context,dtype=torch.float32), LAMBDA, theta)[0]
    assortment = assort(context, profit.tolist()[0], v_array.tolist() )
    purchase_vector = get_linear_purchase(assortment)

    # update the parameters
    LAMBDA = update_A(LAMBDA, assortment)
    
    # update theta using MLE
    
    

In [10]:
context = CONTEXT_ARRAY[1]
profit = REWARD_ARRAY[1]

v_array = calculate_v(torch.tensor(context,dtype=torch.float32), LAMBDA, theta)[0]
ass = assort(context, profit.tolist()[0], v_array.tolist() )